In [ ]:
from spectralmc.sobol_sampler import SobolSampler, BoundSpec
from spectralmc.gbm import SimulationParams, BlackScholes
import pandas as pd
import numpy as np
from typing import Optional, Literal
import plotly.express as px
from math import exp, sqrt, log, erf
import QuantLib as ql
from pathlib import Path

In [ ]:
dtype='float64'

In [ ]:
assert (dtype in ['float32','float64'])
base_dir   = Path("..") / "tests" / ".failed_artifacts"
file_name  = f"bs_mc_failure_{dtype}.parquet"
file_path  = base_dir / file_name
df=pd.read_parquet(file_path)
df

In [ ]:
index=6
inputs=BlackScholes.Inputs(**df.iloc[index]['inputs'].to_dict())
inputs

In [ ]:
sp = SimulationParams(
    timesteps=1,
    network_size=256,
    batches_per_mc_run=(2**20),
    threads_per_block=256,
    mc_seed=42,
    buffer_size=4,
    dtype=dtype,
    simulate_log_return=True,
    normalize_forwards=False,
)
bs = BlackScholes(sp=sp)

In [ ]:
%%time
prices=[bs.price_to_host(inputs) for _ in range(1024)]

In [ ]:
bs._normal_gen.get_time_spent_synchronizing()

In [ ]:
bs._normal_gen.get_idle_time()

In [ ]:
def bs_price_quantlib(inp: BlackScholes.Inputs) -> BlackScholes.HostPricingResults:
    """Closed-form Black price via QuantLib.blackFormula."""
    std = inp.v * sqrt(inp.T)
    disc = exp(-inp.r * inp.T)
    fwd = inp.X0 * exp((inp.r - inp.d) * inp.T)

    call = ql.blackFormula(ql.Option.Call, inp.K, fwd, std, disc)
    put = ql.blackFormula(ql.Option.Put, inp.K, fwd, std, disc)

    cint = disc * max(fwd - inp.K, 0.0)
    pint = disc * max(inp.K - fwd, 0.0)

    return BlackScholes.HostPricingResults(
        call_price_intrinsic=cint,
        put_price_intrinsic=pint,
        underlying=fwd,
        call_convexity=call - cint,
        put_convexity=put - pint,
        call_price=call,
        put_price=put,
    )

In [ ]:
fwd=inputs.X0*exp((inputs.r-inputs.d)*inputs.T)
print(fwd)
print((fwd-inputs.K)*exp(-inputs.r*inputs.T))

In [ ]:
%%time
mc_price=bs.price_to_host(inputs)
mc_price

In [ ]:
analytic=bs_price_quantlib(inputs)
analytic

## Part 3: repeated samplings of an arbitrary point

In [ ]:
%%time
samples=pd.DataFrame([bs.price_to_host(inputs).model_dump() for _ in range(1024)])

In [ ]:
samples_added=samples.copy()
samples_added['parity_check']=np.abs(samples_added['put_convexity']-samples_added['call_convexity'])
samples_added=samples_added.sort_values('parity_check',ascending=False)
samples_added

In [ ]:
px.scatter(samples_added,x='underlying',y='parity_check')

In [ ]:
samples.mean()

In [ ]:
samples.std()

In [ ]:
samples.min()

In [ ]:
samples.max()

In [ ]:
px.violin(samples['put_convexity']-analytic.put_convexity,box=True)